In [1]:
from os.path import split

import numpy as np
import pandas as pd
import csv
from dateutil.parser import parse

In [2]:
ecart = pd.Timedelta(minutes=5)

In [8]:
data = pd.read_csv('data/data_raw.csv' , delimiter=';')
#print(data[['Date','Time']])
date_time = pd.DataFrame()
date_time['date_time'] = pd.to_datetime(data['Time'] +' '+data['Date'])
data['date_time'] =  pd.to_datetime(data['Time'] +' '+data['Date'])
data.drop(columns=['Time' ,'Date','date_time'], inplace=True)
data.insert(0,'Datetime',date_time['date_time'])

formated_data = pd.DataFrame()
for col in data.columns:
    try:
        formated_data[col] = data[col].astype(str).str.replace(',', '.').astype(float)
    except ValueError as e:
        formated_data[col] = data[col]
        print("Erreur de conversion :", e)



print(formated_data)



Erreur de conversion : could not convert string to float: '2024-04-12 23:00:08'
                 Datetime  Breaks  VANNE_UTA_CHAUD [I]  VANNE_UTA_FROID [I]  \
0     2024-04-12 23:00:08     NaN                  0.0                 40.0   
1     2024-04-12 23:00:38     NaN                  0.0                 40.0   
2     2024-04-12 23:01:08     NaN                  0.0                 34.0   
3     2024-04-12 23:01:38     NaN                  0.0                 34.0   
4     2024-04-12 23:02:08     NaN                  0.0                 34.0   
...                   ...     ...                  ...                  ...   
15306 2025-10-01 08:58:32     NaN                 17.0                  0.0   
15307 2025-10-01 08:59:01     NaN                 31.0                  0.0   
15308 2025-10-01 08:59:02     NaN                 31.0                  0.0   
15309 2025-10-01 08:59:31     NaN                 41.0                  0.0   
15310 2025-10-01 08:59:32     NaN                 4

In [4]:
formated_data.to_csv('new_data/data_melun_1.csv' , sep=';', index=False)

In [7]:
def str_to_datetime(date_str):
    try:
        return parse(date_str)  # Essaie d'analyser la date automatiquement
    except (ValueError, TypeError):
        return None


In [11]:

# Lecture du fichier CSV
with open('new_data/data_melun_1.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')

    # Lecture des colonnes (header)
    cols = next(reader, None)

    # Récupération de la première ligne
    first = next(reader)
    first_date = str_to_datetime(first[0])
    print("Première date :", first_date)

    # Initialisation des dictionnaires
    final_data = {col: [] for col in cols}
    test = {col: [] for col in cols}

    #print("Colonnes détectées :", final_data.keys())

    for row in reader:
        date = str_to_datetime(row[0])
        print("Date :", date)
        count = 0
        if date <= first_date + ecart:
            count += 1
            for index, valeur in enumerate(row):
                # Convertir les valeurs en float si possible
                try:
                    valeur = float(valeur) if valeur.replace('.', '', 1).replace('-', '', 1).isdigit() else valeur
                except ValueError:
                    pass  # Garde la valeur en tant que chaîne si elle ne peut pas être convertie

                test[cols[index]].append(valeur)

            #print(f"Ligne ajoutée (index {count}):", test)

        if date >= first_date + ecart:


            # Calculer la moyenne des colonnes numériques
            for col in cols[1:]:  # On saute la colonne "Datetime"
                try:
                    # Convertir les valeurs en float et ignorer les erreurs

                    valeurs = [float(x) for x in test[col] if isinstance(x, (int, float))]
                    print(valeurs)
                    moyenne = sum(valeurs) / len(valeurs) if valeurs else 0
                    print(moyenne)
                    # Vérifier que les moyennes sont bien stockées
                    final_data[col].append(moyenne)

                      # Mise à jour de la première date pour le prochain bloc
                except ValueError:
                    print(f"Impossible de calculer la moyenne pour {col} (valeurs non numériques)")
            final_data['Datetime'].append(first_date)
            first_date = date
            test.clear()
            test = {col: [] for col in cols}

print("<<<<<<<<<<<<<<<<<<< test (Valeurs collectées) >>>>>>>>>>>>>>>>>>>")
#print(test)

print("<<<<<<<<<<<<<<<<<<< final_data (Moyennes finales) >>>>>>>>>>>>>>>>>>>")
print(final_data)


Première date : 2025-02-25 19:05:20
Date : 2025-02-25 19:05:50
Date : 2025-02-25 19:06:20
Date : 2025-02-25 19:06:50
Date : 2025-02-25 19:07:20
Date : 2025-02-25 19:07:50
Date : 2025-02-25 19:08:20
Date : 2025-02-25 19:08:50
Date : 2025-02-25 19:09:20
Date : 2025-02-25 19:09:50
Date : 2025-02-25 19:10:20
[]
0
[69.0, 70.0, 71.0, 71.0, 70.0, 69.0, 70.0, 71.0, 72.0, 72.0]
70.5
[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]
3.0
[0.30000001, 0.5, 0.5, 0.30000001, 0.1, 0.2, 0.30000001, 0.5, 0.40000001, 0.30000001]
0.340000005
[0.1525, 0.1525, 0.1525, 0.1525, 0.1525, 0.126666666666666, 0.126666666666666, 0.126666666666666, 0.126666666666666, 0.126666666666666]
0.139583333333333
[2.6567426, 2.7350135, 2.7742596, 2.7340879, 2.6563225, 2.6374164, 2.6954131, 2.7742596, 2.7930722, 2.772768]
2.72293554
[0.11337893, 0.11337893, 0.11337893, 0.11337893, 0.11337893, 0.11337893, 0.11337893, 0.11337893, 0.11337893, 0.11337893]
0.11337893000000002
Date : 2025-02-25 19:10:50
Date : 2025-02-25 19:11:20


In [12]:
new_data = pd.DataFrame(final_data)
print(new_data)

new_data.to_csv('new_data/new_data_melun_1.csv' , sep=';', index=False)

                Datetime  Breaks  HYGROMETRIE_CELLULE [I]  \
0    2025-02-25 19:05:20       0                70.500000   
1    2025-02-25 19:10:20       0                72.200000   
2    2025-02-25 19:15:20       0                72.400000   
3    2025-02-25 19:20:20       0                72.888889   
4    2025-02-25 19:25:21       0                74.600000   
...                  ...     ...                      ...   
1607 2025-03-03 09:02:49       0                51.600000   
1608 2025-03-03 09:07:49       0                52.000000   
1609 2025-03-03 09:12:49       0                52.800000   
1610 2025-03-03 09:17:49       0                53.100000   
1611 2025-03-03 09:22:50       0                53.800000   

      HYGROMETRIE_CONSIGNE [I]  TEMPERATURE_REPRISE [R]  \
0                          3.0                 0.340000   
1                          3.0                 0.300000   
2                          3.0                 0.160000   
3                          3.0 